<a href="https://colab.research.google.com/github/eTroupe5201/SE489_Group_Project/blob/main/Model_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install hugging face dataset, accelerate and transformers library

In [ ]:
!pip install datasets
!pip install transformers -U
!pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-non

Import rotten tomatoes dataset from huggingface. Data is automatially split for training, validation, and testing.

*   Training split: 8530 examples
*   Validation split: 1066 examples
*   Test split: The remaining examples



In [ ]:
from datasets import load_dataset

# Load the dataset
training_dataset = load_dataset("rotten_tomatoes", split="train")

# Access the validation split
validation_dataset = load_dataset("rotten_tomatoes", split="validation")

# Access the validation split
test_dataset = load_dataset("rotten_tomatoes", split="test")



Initialize tokenizer and model. Utilizing Distilbert model from hugging face and a pre-trained tokenizer. This model has been fine-tuned on the Stanford Sentiment Treebank dataset.

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import load_dataset
# Initialize tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Import pytorch library, training arguments and dataloader
#Define Training Arguments:

*   Learning rate hyperparmeter: is set to 2e-5. It controls how much we are adjusting the model parameters during training.
*   Epochs: set to 1.This is the number of training epochs. It specifies how many times the entire dataset will be passed through.
*   Batch Size: is set to 32. It determines the number of samples processed in each iteraion during training.



In [ ]:
import torch
from torch.utils.data import DataLoader

from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir='./output',                    # Specify the output directory
    per_device_train_batch_size=16,           # Batch size per GPU during training
    num_train_epochs=1,                       # Number of training epochs
    logging_dir='./logs',                     # Directory for logging training metrics
    logging_steps=500,                        # Interval for logging training metrics
    save_steps=1000,                          # Interval for saving model checkpoints
    evaluation_strategy="epoch",              # Evaluation strategy (perform evaluation after each epoch)
    learning_rate=2e-5,                       # Learning rate
)

# You can access the training parameters like this:
learning_rate = training_args.learning_rate
num_epochs = training_args.num_train_epochs
batch_size = training_args.per_device_train_batch_size



In [ ]:
# Define optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)


#PREPROCESS DATA


*   Tokenize Data: tokenize entire dataset
*   Padding: set to true, ensures that sequences have the same length
* Truncating: ensures that sequences are not longer than the maximum length allowed.


Convert label data into Pytourch tensors

Create Pytorch Dataloader


Training Dataset

In [ ]:

# Tokenize and pad the entire dataset
tokenized_dataset = tokenizer(training_dataset['text'], padding=True, truncation=True, return_tensors='pt')

# Convert labels to tensors
labels = torch.tensor(training_dataset['label'])

# Create DataLoader with aligned batch sizes
dataloader = DataLoader(list(zip(tokenized_dataset['input_ids'], tokenized_dataset['attention_mask'], labels)), batch_size=batch_size, shuffle=True)


Validation Dataset

In [ ]:

# Tokenize and pad the entire validation dataset
tokenized_validation_dataset = tokenizer(validation_dataset['text'], padding=True, truncation=True, return_tensors='pt')

# Convert validation labels to tensors
validation_labels = torch.tensor(validation_dataset['label'])

# Create DataLoader for validation dataset with aligned batch sizes
validation_dataloader = DataLoader(
    list(zip(tokenized_validation_dataset['input_ids'], tokenized_validation_dataset['attention_mask'], validation_labels)),
    batch_size=batch_size,
    shuffle=False
)


Test Dataset

In [ ]:
# Tokenize and pad the entire test dataset
tokenized_test_dataset = tokenizer(test_dataset['text'], padding=True, truncation=True, return_tensors='pt')

# Convert test labels to tensors
test_labels = torch.tensor(test_dataset['label'])

# Create DataLoader for test dataset with aligned batch sizes
test_dataloader = DataLoader(
    list(zip(tokenized_test_dataset['input_ids'], tokenized_test_dataset['attention_mask'], test_labels)),
    batch_size=batch_size,
    shuffle=False  # No need to shuffle test data
)


#PREDICTIONS
It is considered good practice predict outcomes before training the model.

torch.no_grad: disables gradient calculation. This speeds up computation and reduces memory consumption.

We unpack the dictionary and compute the output.

logits: raw scores for each class in the classfication task.

torch.argmax(outputs.logits, dim=1) find the index of hte class with the highest confidence socre and specified dimension.

Predictions: contains the predicted class labels for the input samples based on the model's output logits.


In [ ]:
import torch

# Define the mapping between class indices and sentiment labels
#sentiment_mapping = {0: "Negative", 1: "Positive"}

# Initialize a list to store all predictions
all_predictions = []

# Iterate over your dataset in batches
for batch_start in range(0, len(tokenized_test_dataset["input_ids"]), batch_size):
    # Get the current batch
    batch_end = min(batch_start + batch_size, len(tokenized_test_dataset["input_ids"]))
    batch_input_ids = tokenized_test_dataset["input_ids"][batch_start:batch_end]
    batch_attention_mask = tokenized_test_dataset["attention_mask"][batch_start:batch_end]

    # Pass the tokenized text through the model
    with torch.no_grad():
        batch_outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)

    # Interpret the outputs
    batch_predictions = torch.argmax(batch_outputs.logits, dim=1)

    # Append batch predictions to the list
    all_predictions.extend(batch_predictions.tolist())

    # Print progress
    print(f"Processed batch {batch_start // batch_size + 1}/{len(tokenized_test_dataset['input_ids']) // batch_size}")

    # Clear variables to free up memory
    del batch_input_ids, batch_attention_mask, batch_outputs

# Convert the list of predictions to a tensor
all_predictions_tensor = torch.tensor(all_predictions)

# Save all predictions to a file
torch.save(all_predictions_tensor, 'predictions.pt')

print("All predictions saved to 'predictions.pt'")
#save tensor to file


Processed batch 1/66
Processed batch 2/66
Processed batch 3/66
Processed batch 4/66
Processed batch 5/66
Processed batch 6/66
Processed batch 7/66
Processed batch 8/66
Processed batch 9/66
Processed batch 10/66
Processed batch 11/66
Processed batch 12/66
Processed batch 13/66
Processed batch 14/66
Processed batch 15/66
Processed batch 16/66
Processed batch 17/66
Processed batch 18/66
Processed batch 19/66
Processed batch 20/66
Processed batch 21/66
Processed batch 22/66
Processed batch 23/66
Processed batch 24/66
Processed batch 25/66
Processed batch 26/66
Processed batch 27/66
Processed batch 28/66
Processed batch 29/66
Processed batch 30/66
Processed batch 31/66
Processed batch 32/66
Processed batch 33/66
Processed batch 34/66
Processed batch 35/66
Processed batch 36/66
Processed batch 37/66
Processed batch 38/66
Processed batch 39/66
Processed batch 40/66
Processed batch 41/66
Processed batch 42/66
Processed batch 43/66
Processed batch 44/66
Processed batch 45/66
Processed batch 46/

In [ ]:
import torch.nn as nn

# Define the loss function (criterion)
criterion = nn.CrossEntropyLoss()
# Define the print frequency
print_freq = 10  # Print loss every 10 batches

# Training loop
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    print('-' * 10)

    # Set model to training mode
    model.train()

    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for batch_idx, batch in enumerate(dataloader):
        input_ids, attention_mask, batch_labels = batch

        optimizer.zero_grad()

        # Forward pass: compute predicted outputs
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Compute loss
        loss = criterion(logits, batch_labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Update statistics
        running_loss += loss.item() * input_ids.size(0)
        _, predicted = torch.max(logits, 1)
        correct_predictions += (predicted == batch_labels).sum().item()
        total_predictions += batch_labels.size(0)

        if (batch_idx + 1) % print_freq == 0:
            print(f'Batch {batch_idx + 1}/{len(dataloader)}, Loss: {loss.item():.4f}')

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = correct_predictions / total_predictions

    print(f'Epoch {epoch + 1} completed, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')



Epoch 1/1
----------
Batch 10/534, Loss: 0.9674
Batch 20/534, Loss: 0.0041
Batch 30/534, Loss: 0.5640
Batch 40/534, Loss: 0.3962
Batch 50/534, Loss: 0.8621
Batch 60/534, Loss: 0.3809
Batch 70/534, Loss: 0.6248
Batch 80/534, Loss: 0.3085
Batch 90/534, Loss: 0.7445
Batch 100/534, Loss: 0.5448
Batch 110/534, Loss: 0.1859
Batch 120/534, Loss: 0.1369
Batch 130/534, Loss: 0.0036
Batch 140/534, Loss: 0.1237
Batch 150/534, Loss: 0.1918
Batch 160/534, Loss: 0.1146
Batch 170/534, Loss: 0.0029
Batch 180/534, Loss: 1.0011
Batch 190/534, Loss: 0.0078
Batch 200/534, Loss: 0.6805
Batch 210/534, Loss: 0.2548
Batch 220/534, Loss: 0.4049
Batch 230/534, Loss: 1.0168
Batch 240/534, Loss: 0.4992
Batch 250/534, Loss: 0.4389
Batch 260/534, Loss: 0.0009
Batch 270/534, Loss: 0.3539
Batch 280/534, Loss: 0.9450
Batch 290/534, Loss: 0.2469
Batch 300/534, Loss: 0.3162
Batch 310/534, Loss: 0.4576
Batch 320/534, Loss: 0.0041
Batch 330/534, Loss: 0.7940
Batch 340/534, Loss: 0.0024
Batch 350/534, Loss: 0.3944
Batch 36

In [24]:
from torch.utils.data import DataLoader
#data loader for validation set

validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

# Validation loop
model.eval()  # Set the model to evaluation mode
total_correct = 0
total_samples = 0
with torch.no_grad():  # No need to compute gradients during validation
    for batch in validation_dataloader:
        inputs = tokenizer(batch['text'], return_tensors='pt', padding=True, truncation=True)
        labels = batch['label']

        outputs = model(**inputs)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1)

        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)

accuracy = total_correct / total_samples
print(f"Validation Accuracy: {accuracy}")


Validation Accuracy: 0.9015009380863039


In [26]:
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
# Test loop
model.eval()  # Set the model to evaluation mode
total_correct = 0
total_samples = 0
with torch.no_grad():  # No need to compute gradients during testing
    for batch in test_dataloader:
        inputs = tokenizer(batch['text'], return_tensors='pt', padding=True, truncation=True)
        labels = batch['label']

        outputs = model(**inputs)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1)

        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)

accuracy = total_correct / total_samples
print(f"Test Accuracy: {accuracy}")


Test Accuracy: 0.8968105065666041


Write a brief project description, including:
A. The overall goal of the project.

*   To develop a machine learning model for sentiment analysis on movie reviews, with the aim of accurately classifying each review as either positive or negative. By leveraging pre-trained models and fine-tuning techniques, the overall goal is to create a reliable classifier that effectively analyzes large volumes of text data.

B. The framework you plan to use, and how you intend to incorporate it into your project.

*   PyTorch: transformers
*   HuggingFace: Model, Dataset
*   Google Colab
*   TensorFlow


C. The data you initially plan to use (this may change).

*   Hugging Face Dataset: Rotten Tomatoes

D. The models you expect to use.

*   DistilBertForSequenceClassification

